# 准备数据：从文本中构建词向量

## 创建一些实验样本

1. postingList: 进行词条切割后的文档集合，这相当于是六个文档  
2. classVec：类别标签的集合

In [1]:
from numpy import *

def loadDataSet():
    postingList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love, 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0,1,0,1,0,1]    #1 is abusive, 0 not
    return postingList,classVec

## 创建一个包含所有文档中出现的单词列表(一个不重复的列表)

In [2]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document) #union of the two sets
    return list(vocabSet)

## 输出词汇表中的单词在文档中是否出现过

In [4]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)  # 把myVocabList变成一个向量
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1  # 只要文档（inputSet）中出现的单词都标记一下
        else: print ("the word: %s is not in my Vocabulary!" % word)
    return returnVec

## 测试

In [13]:
listOPosts, listClasses = loadDataSet()  # 创建训练样本
myVocabList = createVocabList(listOPosts)  # 把loadDataSet中postingList中的所有单词放在一起创建了一个不重复的词汇表
setOfWords2Vec(myVocabList, listOPosts[0])  # listOPostsP[0]是postingList的第一行；

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1]

In [65]:
myVocabList

['my',
 'so',
 'worthless',
 'food',
 'ate',
 'him',
 'to',
 'garbage',
 'how',
 'buying',
 'cute',
 'mr',
 'dog',
 'please',
 'maybe',
 'steak',
 'licks',
 'is',
 'not',
 'flea',
 'stupid',
 'park',
 'posting',
 'love',
 'help',
 'problems',
 'dalmation',
 'stop',
 'I',
 'quit',
 'take',
 'has']

# 训练算法：根据词向量计算概率

## 朴素贝叶斯分类器训练函数

1. trainMatrix：文档矩阵  
2. trainCategory：每篇文档类别标签所构成的向量 [0, 1, 0, 1, 0, 1]

In [59]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)  # 几个文档，6
    numWords = len(trainMatrix[0])  # 几个单词
    pAbusive = sum(trainCategory) / float(numTrainDocs)  # 3 / 6  含有侮辱性词语的文档/总文档数目
    p0Num = zeros(numWords)
    p1Num = zeros(numWords)
    p0Denom = 0.0
    p1Denom = 0.0  
    for i in range(numTrainDocs):  
        if trainCategory[i] == 1:  # 当前文档是不是含有侮辱性词汇 
            p1Num += trainMatrix[i]  # 把所有标记为侮辱性文档的词都记录在一起
            p1Denom += sum(trainMatrix[i]) # 标记为侮辱性文档的总词数
        else:
            p0Num += trainMatrix[i]  # 把所有标记为侮辱性文档的词都记录在一起
            p0Denom += sum(trainMatrix[i])  # 标记为正常文档的总词数
    p1Vect = p1Num / p1Denom 
    p0Vect = p0Num / p0Denom
    return p0Vect, p1Vect, pAbusive

## 测试

1. listOPosts：文档数据  
2. postinDoc：listOPosts中的每一行  
3. listClasses：类别数据  
4. listClasses：词汇表  
5. trainMat：由文档中单词是否出现的词汇表构成，因为文档有六行（即6篇文档），所以trainMat也有六行

In [60]:
listOPosts, listClasses = loadDataSet()  # 创建训练样本
myVocabList = createVocabList(listOPosts)  # 把loadDataSet中postingList中的所有单词放在一起创建了一个不重复的词汇表
trainMat = []  # 文档矩阵
for postinDoc in listOPosts:  # 一行一行的看单词是否在词汇表中出现过
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))  # 把每一行的单词出现情况加入到trainMat列表中，形成一个矩阵
len(trainMat)

6

In [61]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

In [62]:
p0V

array([0.125     , 0.04166667, 0.        , 0.        , 0.04166667,
       0.08333333, 0.04166667, 0.        , 0.04166667, 0.        ,
       0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.        ,
       0.04166667, 0.04166667, 0.04166667, 0.        , 0.04166667,
       0.        , 0.        , 0.        , 0.04166667, 0.04166667,
       0.04166667, 0.04166667, 0.04166667, 0.04166667, 0.        ,
       0.        , 0.04166667])

In [63]:
p1V

array([0.        , 0.        , 0.10526316, 0.05263158, 0.        ,
       0.05263158, 0.05263158, 0.05263158, 0.        , 0.05263158,
       0.        , 0.        , 0.10526316, 0.        , 0.05263158,
       0.        , 0.        , 0.        , 0.05263158, 0.        ,
       0.15789474, 0.05263158, 0.05263158, 0.        , 0.        ,
       0.        , 0.        , 0.05263158, 0.        , 0.05263158,
       0.05263158, 0.        ])

In [64]:
pAb

0.5

In [68]:
argmax(p1V)  # 侮辱性文档中出现次数最多的单词下标

20

In [70]:
myVocabList[argmax(p1V)]  # 侮辱性文档中出现次数最多的单词

'stupid'

# 测试算法：根据现实情况修改分类器

#### 修改  
1. 避免在计算某个文档属于某个类别的概率的时候（要进行连乘操作）出现概率为0的情况  
p0Num = ones(numWords)  
p1Num = ones(numWords)    
p0Denom = 2.0  
p1Denom = 2.0    
2. 避免下溢,对乘积取自然对数  
p1Vect = log(p1Num / p1Denom) 
p0Vect = log(p0Num / p0Denom)  

#### 注
1. 西瓜书中使用的办法是拉普拉斯修正  
分子统一加1  
分母统一加第i个属性的可能取值数  

In [74]:
def trainNB01(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)  # 几个文档，6
    numWords = len(trainMatrix[0])  # 几个单词
    pAbusive = sum(trainCategory) / float(numTrainDocs)  # 3 / 6  含有侮辱性词语的文档/总文档数目
    p0Num = ones(numWords)
    p1Num = ones(numWords)  
    p0Denom = 2.0
    p1Denom = 2.0  
    for i in range(numTrainDocs):  
        if trainCategory[i] == 1:  # 当前文档是不是含有侮辱性词汇 
            p1Num += trainMatrix[i]  # 把所有标记为侮辱性文档的词都记录在一起
            p1Denom += sum(trainMatrix[i]) # 标记为侮辱性文档的总词数
        else:
            p0Num += trainMatrix[i]  # 把所有标记为侮辱性文档的词都记录在一起
            p0Denom += sum(trainMatrix[i])  # 标记为正常文档的总词数
    p1Vect = log(p1Num / p1Denom)  
    p0Vect = log(p0Num / p0Denom)  
    return p0Vect, p1Vect, pAbusive

## 朴素贝叶斯分类函数

1. vec2Classify：要分类的向量  
2. p0Vec, p1Vec, pClass1：trainNB01计算得到的三个概率

In [75]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + log(pClass1)    #element-wise mult
    p0 = sum(vec2Classify * p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else: 
        return 0

## 便利函数

封装所有的操作

In [78]:
def testingNB():
    listOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(listOPosts)
    trainMat=[]
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
    p0V,p1V,pAb = trainNB0(array(trainMat),array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))
    testEntry = ['stupid', 'garbage']
    thisDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(testEntry,'classified as: ',classifyNB(thisDoc,p0V,p1V,pAb))

In [80]:
testingNB()

['love', 'my', 'dalmation'] classified as:  0
['stupid', 'garbage'] classified as:  1
